# A Markov chain monte carlo example

In the lecures we saw how to calculate a posterior distribution using Bayes' theorem. Remember, that for a continuous parameter $\theta$, 

$$
p(\theta|X) = \frac{p(X|\theta)p(\theta)}{p(X)}
$$

where $X$ represents our data, $p(\theta)$ is the *prior* probabillity of $\theta$, and $p(X)=\int p(\theta|X)d\theta$. 

To make statistical statements about the parameter $\theta$, we need to know the posterior probabity distribution $p(\theta|X)$. It is often the case (when the dataset is large or the probability model complex) that rather than calculate the posterior directly, we use monte carlo simulations to extract properties of the posterior distribution. 

In [ ]:
#%matplotlib notebook

import numpy
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})

Let's take a simple example where $X\sim \phi(\theta,1)$ - is normally distributed and $p(\theta)=phi(0.5,2)$. Suppose our data is represented by a single measurement of $X=1.2$.

Our prior is now 

$$
p(\theta)\propto e^{-\frac{1}{2}\frac{(\theta-0.5)^{2}}{2^{2}}}
$$

and the *likelihood*  is, 

$$
p(X|\theta)\propto e^{-\frac{1}{2}\frac{(1.2-\theta)^{2}}{1^{2}}}
$$

where we've ignored  the normalisation terms. 

We can quickly calculate the posterior probability using the `scipy.stats.norm.pdf` method (which yields the probability density of a normal distribution) and since we need to integrate, we'll use the numerical integration `scipy.integrate`.

In [ ]:
from scipy import stats
import scipy.integrate as integrate

xmeas = 1.2

def prior(th): 
    return stats.norm.pdf(th,0.5,2)

def likelihood(th):
    return stats.norm.pdf(xmeas,th,1)

def posterior(th):
    return likelihood(th)*prior(th)

pX = integrate.quad(posterior,-8,8)[0]

thvals = numpy.linspace(-5,5,100)
post   = [posterior(th)/pX for th in thvals]
plt.plot(thvals,post,color='red')
plt.xlabel("$\\theta$")
plt.ylabel("$p(\\theta|X)$")

plt.show()

## A basic MC approach

We can see that (as expected) the posterior is centered around our measured value of $X=1.2$. 

We could obtain the same distribution using monte carlo simulation as follows: 

   * First we randomly pick values of $\theta\rightarrow \theta'$, according to our prior $p(\theta)$. 
   * Decide whether to keep the value generated according to the likelihood. 
       * Generate a uniform random value $r\sim U(0,1)$
       * Keep the generated value of $\theta$ if $r<p(X|\theta\prime)$ 

In [ ]:
mcprior = numpy.random.normal(0.5,2,10000)

# I'm using a for loop in python! It's ok because it makes the steps easy to read
accepted_mc = []
for mcp in mcprior :
    rnd = numpy.random.uniform(0,1)
    plike = likelihood(mcp)
    if rnd < plike: accepted_mc.append(mcp)

plt.hist(accepted_mc,density=True,fill=False,edgecolor='blue')
plt.plot(thvals,post,color='red')
plt.xlabel("$\\theta$")
plt.ylabel("$p(\\theta|X)$")
plt.show()

The resulting distribution of values which are kept approximates the posterior distribution, as can be seen by the fact that the red line (the posterior distribution we calcuated before) matches the histogram. 

From the passing samples, we can calculate things like the mean of the posterior, or quantile ranges, that can be used for making statements about $\theta$. 

Note however, that although we generated 10,000 samples,  the number of entries  in the histogram is much less due to the acceptance step. In fact, the efficiency of the acceptance is quite low.

In [ ]:
print("fraction of accepted samples",  float(len(accepted_mc))/len(mcprior))

This would be very inefficient for large dimensional data spaces or more complicated likelihoods. Thankfully we can improve things using a  markov chain approach..

## A Markov chain approach and Metropolis-Hastings

A Markov chain MC is any MC proccess where each step is dependent on the current position. 

In our simple MC, each step was completely independent of one another (the random values of $\theta$ were independent). 

In the Metropolis-Hastings algorithm, we modify the algorithm to accept or reject a  proposed value for the parameter based on the ratios of the posterior probability between the current value of the parameter ($\theta$) and the proposed one ($\theta_{\text{new}}$). We define the ratio, 

$$
r(\theta_{\text{new}},\theta) = \frac{p(\theta_{\text{new}}|X)}{p(\theta|X)} = \frac{p(X|\theta_{\text{new}})p(\theta_{\text{new}})}{p(X|\theta)p(\theta)}
$$

Note that there is no need to normalise this since the normalisation term $p(X)$ cancels in the ratio. 

The decision to accept or reject the value will be determined by the value of $r$. 

  * if $r>1$ - accept $\theta_{new}$ 
  * else 
     * define $alpha = min(r,1)$
     * draw a random uniform value between 0 and 1 , $t\sim U(0,1)$. 
     * if $t<\alpha$, accept $\theta_\text{new}$ as the new value of $\theta$, otherwise reject it and keep $\theta$ as the value. 
     
The idea is that the chain will tend to move towards higher values of the posterior such that any infrence we peform will be more accurate as most of our samples will be around the maximum of the posterior. 

In [ ]:
# define our proposal generator
# a common choice is a Gaussian centered at the current value of theta
def proposal(th):
    return numpy.random.normal(th,1)

# function to accept new proposal or reject it 
def accept_or_reject(th_new,th):  
    r = posterior(th_new)/posterior(th)
    if r > 1: 
        return th_new,True 
    else:
        alpha = min([r,1])
        t = numpy.random.uniform(0,1)
        if t < alpha : 
            return th_new,True
        else: return th,False

init_th = proposal(3)
accepted_mc_MH = []

number_steps = 10000
for i in range(number_steps) :
    
    th_new = proposal(init_th)
    th_new,accept = accept_or_reject(th_new,init_th)
    init_th = th_new
    if accept: accepted_mc_MH.append(th_new)
        
# plot the accepted points vs the step
plt.plot(range(len(accepted_mc_MH)),accepted_mc_MH)

Its usually a good idea to remove some first chunk of the markov chain since the initial part will depend on the initial value and the proposal function, meaning the distribution of the samples changes as the process continues. This is known as the *burn-in* period. We can just throw away the first 1000 points to remove this part. 

In [ ]:
accepted_mc_MH = accepted_mc_MH[1000:-1]

And now plot the results. Again, we can compare to the analytic version.

In [ ]:
plt.hist(accepted_mc_MH,density=True,fill=False,edgecolor='green',bins=25,label='Markov chain (MH)')
plt.plot(thvals,post,color='red',label='Analytic')
plt.legend()
plt.xlabel("$\\theta$")
plt.ylabel("$p(\\theta|X)$")
plt.savefig("markovchainexample.pdf")
plt.show()

Note again that the resulting distribution resembles the direct calculation. However this time, we have many more samples in the distribution since the generation is more efficient. 

In [ ]:
print("fraction of accepted samples - simple MC",  float(len(accepted_mc))/len(mcprior))
print("fraction of accepted samples - MC MH",  float(len(accepted_mc_MH))/number_steps)

It's usually a good idea to check that the calculation has converged. One way to check is to look at the history of the parameter values - in our case it looks like the chain doesn't wander into some bizarre region of the parameter values so seems ok. 

We can also check if the earlier part of the samples looks the same as the latter parts. For example, lets compare the first half of samples after the burn-in to the second half. 

In [ ]:
halfpoint = int(len(accepted_mc_MH)/2)
plt.hist(accepted_mc_MH[0:halfpoint],density=True,
         fill=True,edgecolor='blue',bins=25,range=[-2,5],label='1st half')
plt.hist(accepted_mc_MH[halfpoint:-1],density=True,
         fill=False,edgecolor='red',bins=25,range=[-2,5],label='2nd half')
plt.legend()
plt.show()

Of course, there are several packages out there that will perform markov chain mc for us based in `python`, `R`, or `c++` (eg see `pymc`, `MarkovChainMC` in `RooFit`...) etc. These will have routines for you to make these checks. 